In [ ]:
#Basic Client
import socket
import threading


def handler(sock):
    
    while True:
        data = sock.recv(1024)
        print('\nreceived: ',data.decode())
        

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = input("Server Address(default=127.0.0.1): ")
if host == '':
    host = '127.0.0.1'
sock.connect((host, 2500))
cThread = threading.Thread(target=handler, args=(sock,))
cThread.daemon = True
cThread.start()

while True:
    msg = input('> ')
    sock.send(msg.encode())


received:  ASD

received:  DDD


In [ ]:
import socket
import threading

def handler(sock):
    while True:
        try:
            data = sock.recv(1024).decode()
            print(data)
        except:
            print("Disconnected from server")
            break

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = input("Server Address(default=127.0.0.1): ")
if host == '':
    host = '127.0.0.1'
sock.connect((host, 2500))

# 닉네임 입력 및 전송
nickname = input("Enter your nickname: ")
sock.send(nickname.encode())

# 서버로부터 환영 메시지 수신
welcome_msg = sock.recv(1024).decode()
print(welcome_msg)

cThread = threading.Thread(target=handler, args=(sock,))
cThread.daemon = True
cThread.start()

while True:
    msg = input()
    if msg.lower() == 'quit':
        break
    sock.send(msg.encode())

sock.close()
print("Disconnected from server")


Welcome! Please enter your nickname: 
1 has joined the chat!


In [ ]:
import socket
import threading
import tkinter as tk
from tkinter import ttk, simpledialog

class ChatClient:
    def __init__(self, master):
        self.master = master
        self.master.title("NA_KakaoTalk")
        self.master.geometry("300x500")
        self.master.configure(bg="#b2c7d9")

        self.nickname = self.get_nickname()

        self.chat_canvas = tk.Canvas(master, bg="#b2c7d9", highlightthickness=0)
        self.chat_canvas.pack(expand=True, fill='both', padx=10, pady=10)

        self.scrollbar = ttk.Scrollbar(self.chat_canvas, orient="vertical", command=self.chat_canvas.yview)
        self.scrollbar.pack(side="right", fill="y")

        self.chat_canvas.configure(yscrollcommand=self.scrollbar.set)

        self.message_frame = tk.Frame(self.chat_canvas, bg="#b2c7d9")
        self.chat_canvas.create_window((100, 0), window=self.message_frame, anchor="nw")

        self.input_frame = tk.Frame(master, bg="#b2c7d9")
        self.input_frame.pack(side=tk.BOTTOM, fill=tk.X, padx=10, pady=10)

        self.message_entry = tk.Entry(self.input_frame, bg="white", fg="black")
        self.message_entry.pack(side=tk.LEFT, expand=True, fill=tk.X, padx=(0, 5))
        self.message_entry.bind("<Return>", self.send_message)

        self.send_button = ttk.Button(self.input_frame, text="전송", command=self.send_message)
        self.send_button.pack(side=tk.RIGHT)

        self.message_frame.bind("<Configure>", self.on_frame_configure)

        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.connect_to_server()

    def get_nickname(self):
        return simpledialog.askstring("Nickname", "Enter your nickname:", parent=self.master)

    def on_frame_configure(self, event):
        self.chat_canvas.configure(scrollregion=self.chat_canvas.bbox("all"))

    def connect_to_server(self):
        host = simpledialog.askstring("Server Address", "Enter server address:", parent=self.master) or '127.0.0.1'
        try:
            self.sock.connect((host, 2500))
            self.sock.send(self.nickname.encode())
            welcome_msg = self.sock.recv(1024).decode()
            self.display_message(welcome_msg, 'left', 'white')
            receive_thread = threading.Thread(target=self.receive_messages)
            receive_thread.daemon = True
            receive_thread.start()
        except Exception as e:
            self.display_message(f"Connection error: {str(e)}", 'left', 'red')

    def send_message(self, event=None):
        message = self.message_entry.get()
        if message:
            try:
                self.sock.send(message.encode())
                self.display_message(f"{self.nickname}: {message}", 'right', '#fee500')
                self.message_entry.delete(0, tk.END)
            except:
                self.display_message("Failed to send message", 'left', 'red')

    def receive_messages(self):
        while True:
            try:
                data = self.sock.recv(1024)
                if data:
                    self.display_message(data.decode(), 'left', 'white')
            except:
                self.display_message("Disconnected from server", 'left', 'red')
                break

    def display_message(self, message, align, bg_color):
        frame = tk.Frame(self.message_frame, bg="#b2c7d9")
        frame.pack(side=tk.TOP, fill=tk.X, padx=10, pady=5)

        if align == 'left':
            frame.pack_configure(anchor='w')
        else:
            frame.pack_configure(anchor='e')

        msg_label = tk.Label(frame, text=message, bg=bg_color, fg="black", wraplength=200, justify=tk.LEFT, padx=5, pady=5)
        msg_label.pack(side=align)

        self.chat_canvas.update_idletasks()
        self.chat_canvas.yview_moveto(1.0)

if __name__ == "__main__":
    root = tk.Tk()
    client = ChatClient(root)
    root.mainloop()


Exception in thread Thread-10 (receive_messages):
Traceback (most recent call last):
  File "C:\Users\fun67\AppData\Local\Temp\ipykernel_39748\1567126764.py", line 75, in receive_messages
  File "C:\Users\fun67\AppData\Local\Temp\ipykernel_39748\1567126764.py", line 81, in display_message
  File "c:\Users\fun67\anaconda3\Lib\tkinter\__init__.py", line 3224, in __init__
    Widget.__init__(self, master, 'frame', cnf, {}, extra)
  File "c:\Users\fun67\anaconda3\Lib\tkinter\__init__.py", line 2659, in __init__
    self.tk.call(
RuntimeError: main thread is not in main loop

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\fun67\anaconda3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "c:\Users\fun67\anaconda3\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\fun67\AppData\Local\Temp\ipykernel_39748\1567126764.py", line 77, in receive_messages
 

In [1]:
import socket
import threading
import tkinter as tk
from tkinter import ttk, simpledialog, messagebox


class LoginWindow:
    def __init__(self, master):
        self.master = master
        self.master.title("NA_KakaoTalk 로그인")
        self.master.geometry("300x150")
        self.master.configure(bg="#b2c7d9")

        self.frame = tk.Frame(self.master, bg="#b2c7d9")
        self.frame.pack(expand=True, fill='both', padx=20, pady=20)

        self.username_label = tk.Label(self.frame, text="사용자명:", bg="#b2c7d9")
        self.username_label.grid(row=0, column=0, sticky="e", pady=5)
        self.username_entry = tk.Entry(self.frame)
        self.username_entry.grid(row=0, column=1, pady=5)

        self.password_label = tk.Label(self.frame, text="비밀번호:", bg="#b2c7d9")
        self.password_label.grid(row=1, column=0, sticky="e", pady=5)
        self.password_entry = tk.Entry(self.frame, show="*")
        self.password_entry.grid(row=1, column=1, pady=5)

        self.login_button = ttk.Button(self.frame, text="로그인", command=self.login)
        self.login_button.grid(row=2, column=0, columnspan=2, pady=10)


    def login(self):
        username = self.username_entry.get()
        password = self.password_entry.get()
    
        if username and password:
            try:
                if authenticate_with_server(username, password):
                    self.master.destroy()
                    self.open_chat_window(username)
                else:
                    messagebox.showerror("로그인 실패", "잘못된 사용자명 또는 비밀번호입니다.")
            except Exception as e:
                messagebox.showerror("연결 오류", f"서버에 연결할 수 없습니다: {str(e)}")
        else:
            messagebox.showerror("로그인 실패", "사용자명과 비밀번호를 입력하세요.")
        
    def authenticate_with_server(username, password):
        server_host = '127.0.0.1'  # 서버 주소
        server_port = 2500         # 서버 포트

        try:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client_socket:
                client_socket.connect((server_host, server_port))
                credentials = f"{username}:{password}"
                client_socket.send(credentials.encode())
                response = client_socket.recv(1024).decode()
                return response == "True"
        except Exception as e:
            raise Exception(f"서버 연결 오류: {str(e)}")



    def open_chat_window(self, nickname):
        chat_window = tk.Tk()
        ChatClient(chat_window, nickname)


class ChatClient:
    def __init__(self, master, nickname):
        self.master = master
        self.master.title("NA_KakaoTalk")
        self.master.geometry("300x500")
        self.master.configure(bg="#b2c7d9")

        self.nickname = nickname

        self.chat_frame = tk.Frame(master, bg="#b2c7d9")
        self.chat_frame.pack(expand=True, fill='both', padx=10, pady=10)

        self.chat_canvas = tk.Canvas(self.chat_frame, bg="#b2c7d9", highlightthickness=0)
        self.chat_canvas.pack(side="left", expand=True, fill='both')

        self.scrollbar = ttk.Scrollbar(self.chat_frame, orient="vertical", command=self.chat_canvas.yview)
        self.scrollbar.pack(side="right", fill="y")

        self.chat_canvas.configure(yscrollcommand=self.scrollbar.set)

        self.message_frame = tk.Frame(self.chat_canvas, bg="#b2c7d9")
        self.message_frame_window = self.chat_canvas.create_window((0, 0), window=self.message_frame, anchor="nw", width=self.chat_canvas.winfo_width())

        self.input_frame = tk.Frame(master, bg="#b2c7d9")
        self.input_frame.pack(side=tk.BOTTOM, fill=tk.X, padx=10, pady=10)

        self.message_entry = tk.Entry(self.input_frame, bg="white", fg="black")
        self.message_entry.pack(side=tk.LEFT, expand=True, fill=tk.X, padx=(0, 5))
        self.message_entry.bind("<Return>", self.send_message)

        self.send_button = ttk.Button(self.input_frame, text="전송", command=self.send_message)
        self.send_button.pack(side=tk.RIGHT)

        self.chat_canvas.bind("<Configure>", self.on_canvas_configure)
        self.message_frame.bind("<Configure>", self.on_frame_configure)

        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.connect_to_server()

    def on_canvas_configure(self, event):
        self.chat_canvas.itemconfig(self.message_frame_window, width=event.width)

    def on_frame_configure(self, event):
        self.chat_canvas.configure(scrollregion=self.chat_canvas.bbox("all"))
        self.chat_canvas.yview_moveto(1.0)


    def connect_to_server(self):
        host = simpledialog.askstring("Server Address", "Enter server address:", parent=self.master) or '127.0.0.1'
        try:
            self.sock.connect((host, 2500))
            self.sock.send(self.nickname.encode())
            welcome_msg = self.sock.recv(1024).decode()
            self.display_message(welcome_msg, 'left', 'white')
            receive_thread = threading.Thread(target=self.receive_messages)
            receive_thread.daemon = True
            receive_thread.start()
        except Exception as e:
            self.display_message(f"Connection error: {str(e)}", 'left', 'red')

    def send_message(self, event=None):
        message = self.message_entry.get()
        if message:
            try:
                self.sock.send(message.encode())
                self.display_message(f"{self.nickname}: {message}", 'right', '#fee500')
                self.message_entry.delete(0, tk.END)
            except:
                self.display_message("Failed to send message", 'left', 'red')

    def receive_messages(self):
        while True:
            try:
                data = self.sock.recv(1024)
                if data:
                    self.display_message(data.decode(), 'left', 'white')
            except:
                self.display_message("Disconnected from server", 'left', 'red')
                break
    def display_message(self, message, align, bg_color):
        frame = tk.Frame(self.message_frame, bg="#b2c7d9")
        frame.pack(side=tk.TOP, fill=tk.X, padx=10, pady=5)

        if align == 'left':
            frame.pack_configure(anchor='w')
        else:
            frame.pack_configure(anchor='e')

        msg_label = tk.Label(frame, text=message, bg=bg_color, fg="black", wraplength=200, justify=tk.LEFT, padx=5, pady=5)
        msg_label.pack(side=align)

        self.master.update_idletasks()
        self.chat_canvas.yview_moveto(1.0)
if __name__ == "__main__":
    root = tk.Tk()
    login_window = LoginWindow(root)
    root.mainloop()


In [2]:
import tkinter as tk
from tkinter import messagebox
import socket

class LoginTest:
    def __init__(self, master):
        self.master = master
        self.master.title("Login Test")
        
        self.username_entry = tk.Entry(master)
        self.username_entry.pack()
        
        self.password_entry = tk.Entry(master, show="*")
        self.password_entry.pack()
        
        self.login_button = tk.Button(master, text="Login", command=self.login)
        self.login_button.pack()

    def login(self):
        username = self.username_entry.get()
        password = self.password_entry.get()
    
        if username and password:
            try:
                if self.authenticate_with_server(username, password):
                    print("로그인 성공")
                    self.master.destroy()
                else:
                    messagebox.showerror("로그인 실패", "잘못된 사용자명 또는 비밀번호입니다.")
            except Exception as e:
                messagebox.showerror("연결 오류", f"서버에 연결할 수 없습니다: {str(e)}")
        else:
            messagebox.showerror("로그인 실패", "사용자명과 비밀번호를 입력하세요.")
        
    def authenticate_with_server(self, username, password):
        server_host = '127.0.0.1'  # 서버 주소
        server_port = 2500         # 서버 포트

        try:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client_socket:
                client_socket.connect((server_host, server_port))
                credentials = f"{username}:{password}"
                client_socket.send(credentials.encode())
                response = client_socket.recv(1024).decode()
                return response == "True"
        except Exception as e:
            raise Exception(f"서버 연결 오류: {str(e)}")

if __name__ == "__main__":
    root = tk.Tk()
    login_test = LoginTest(root)
    root.mainloop()


로그인 성공
